# Extract Factors
Takes the `points_table.json` file as an input and computes the a best fit quadratic for each event mapping time or performance to the number of points and saving the fit parameters along with the event info in `fit_params.json`

In [5]:
import json
import numpy as np

def fit_function(x, a, ref, shift=0):
    return a*(x + ref)**2 + shift

def loss(params, x_data, y_data):
    a, ref, shift = params
    y_pred = fit_function(x_data, a, ref, shift)
    return np.sum(np.abs(np.floor(y_pred) - np.floor(y_data)))

def time_to_seconds(time_str):
    parts = time_str.split(':')
    seconds = float(parts[-1])
    for i, part in enumerate(reversed(parts[:-1])):
        seconds += int(part) * 60 ** (i+1)
    if '.' not in time_str:
        return seconds - 0.5
    else:
        return seconds - 0.005

points_table = json.load(open("2025_points_table.json", "r"))
points = np.array(points_table["Points"], dtype=float) + 0.5

def get_fit(gender, event):
    times = np.array(points_table[gender][event]['Scores'])
    times = np.array([time_to_seconds(t) if t != "-" else np.nan for t in times])
    print(f"{event}: {times[~np.isnan(times)][-1]}")
    # curve fitting
    x_data = times[~np.isnan(times)]
    y_data = points[~np.isnan(times)]
    a, b, c = np.polyfit(x_data, y_data, 2)
    ref = b / (2 * a)
    shift = c - b**2 / (4 * a)
    # calculate the loss
    error = loss([a, ref, shift], x_data, y_data)
    print(f"{event}: a={a}, ref={ref}, shift={shift}, error={error}")
    return a, ref, shift

In [6]:
fit_params = {"Men": {},
              "Women": {}}

for gender in fit_params.keys():
    for event in points_table[gender].keys():
        fits = get_fit(gender, event)
        fit_params[gender][event] = {}
        fit_params[gender][event]["Fits"] = fits
        fit_params[gender][event]["Distance"] = points_table[gender][event]['Distance']
        fit_params[gender][event]["Category"] = points_table[gender][event]['Category']
        fit_params[gender][event]["Surface"] = points_table[gender][event]['Surface']
        fit_params[gender][event]["Mixed"] = points_table[gender][event]['Mixed']

# save the fit parameters to a JSON file
with open("2025_fit_params.json", "w") as f:
    json.dump(fit_params, f, indent=4)


50m: 9.084999999999999
50m: a=95.82235386298231, ref=-9.194401087362442, shift=0.12340216835855244, error=6.0
55m: 9.875
55m: a=78.92276088460525, ref=-9.9941682962597, shift=0.17374067831315188, error=5.0
60m: 10.565
60m: a=68.62032200466447, ref=-10.69428001635123, shift=0.14057243181832746, error=12.0
100m: 16.785
100m: a=24.64221166330608, ref=-16.992531558274376, shift=0.28133795200028544, error=12.0
200m: 35.044999999999995
200m: a=5.0833296257072185, ref=-35.48611159736082, shift=0.5957410214650736, error=12.0
200m sh: 35.544999999999995
200m sh: a=5.042898433343419, ref=-35.98671911026437, shift=0.5826109375339001, error=22.0
300m: 56.455
300m: a=1.829657024743583, ref=-57.192688143382135, shift=0.4446414074718632, error=9.0
300m sh: 57.245
300m sh: a=1.8029639297594904, ref=-57.990436425910616, shift=0.48213317584122706, error=0.0
400m: 78.00500000000001
400m: a=1.0210130425533175, ref=-78.98969305639112, shift=0.502988005183397, error=0.0
400m sh: 79.58500000000001
400m sh: a